# Klątwa wymiaru
Due: October 25, 2018, 11:45 pm

Celem zadania jest zbadanie, jak w zależności od liczby wymiarów zmieniają się poniższe wartości. Na potrzeby zadania stosujemy odległość Euklidesa.

**Wykonanie: Marcin Przewięźlikowski**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = [10,5]
import pandas as pd
from typing import Tuple

## Objętość hiperkuli vs objętość hipersześcianu
**Mamy hiperkulę o promieniu równym X wpisaną w hipersześcian o krawędziach długości 2X. Hiperkulę w przestrzeniach wielowymiarowych definiujemy jako zbiór punktów o odległości od jej środka nie większej niż jej promień. Zapełniamy hipersześcian losowymi punktami o równomiernym rozkładzie. Jaki % z tych punktów znajdzie się wewnątrz kuli, a jaki na zewnątrz - w "narożnikach"?**


In [ ]:
def difference(x1: np.ndarray, x2: np.ndarray) -> np.ndarray:
    return np.sqrt(
        ((x1 - x2) **2).sum(axis=1)
    )

In [ ]:
def hyper_percentage(degree: int, X:float = 1.0, n_samples: int = 10000) -> float:
    sphere_center = np.ones((1, degree)) * X
    points = np.random.rand(n_samples, degree) * 2*X
    differences = difference(sphere_center, points)
    points_in_ball = (differences < X)
    return points_in_ball.sum() / points_in_ball.shape[0] 

In [ ]:
degrees = np.arange(1, 30)
percentages = [hyper_percentage(degree=d, X=20.0) for d in degrees]

df = pd.DataFrame({
    "degree": degrees,
    "percentage": percentages
})
df

In [ ]:
plt.scatter(df["degree"], df["percentage"])

#### Komentarz

Jak widac z wykresu, liczba punktów trafiającyh do hiperkuli maleje ze wzrostem przyjętego wymiaru. Używamy tu metody Monte Carlo, więc zwiększenie liczby próbek na pomiar pomaga uzyskać dokładniejsze wyniki, jednak trend jest wyraźnie obserwowalny.

Wniosek z tego jest taki, że im większy wymiar danych o rozkładzie jednostajnym, tym mniejsza szansa, że dla danego punktu w tych danych, znajdziemy inny punkt, który będzie mu relatywnie (euklidesowo) bliski.

## Odległości między punktami w hipersześcianie
**Mamy hipersześcian o krawędziach długości 1. Zapełniamy go losowymi punktami o równomiernym rozkładzie. Jaki jest stosunek odchylenia standardowego odległości między tymi punktami do średniej odległości między nimi?**

In [ ]:
def difference_matrix_naive(points: np.ndarray) -> np.ndarray:
    return np.array([difference(p, points) for p in points])

In [ ]:
# implementacja zwektoryzowana
def difference_matrix(points: np.ndarray) -> np.ndarray:
    return  (
        ((points ** 2).sum(axis=1) - 2 * (points @ points.T)).T + 
        (points ** 2).sum(axis=1)
    )

In [ ]:
def std_to_avg_difference(degree: int, X: float=1.0, n_samples: int = 1000) -> float:
    points = (np.random.rand(n_samples, degree) * X)
    dif_mat = difference_matrix(points)
    return [dif_mat.std(), dif_mat.mean()]

In [ ]:
degrees = np.arange(1, 250)
std_avgs = np.array([std_to_avg_difference(degree=d, X=20.0) for d in degrees])

df = pd.DataFrame({
    "degree": degrees,
    "std": std_avgs[:, 0],
    "avg": std_avgs[:, 1],
})
df["std_to_avg"] = df["std"] / df["avg"]
df

In [ ]:
plt.plot(df["degree"], df["std"], color="blue")
plt.plot(df["degree"], df["avg"], color="red")
plt.show()

In [ ]:
plt.scatter(df["degree"], df["std_to_avg"])

#### Komentarz

Stosunek odchylenia standardowego odległości miedzy punktami do średniej odległosci maleje wraz z wymiarem przestrzeni, w której te punkty się znajdują. Im większy wymiar, tym mniej rozkład jednostajny rozrzutu punktów jest podatny na różne fluktuacje, które mogłyby powodować, że niektóre punkty wylądują bliżej siebie. 

W niskich wymiarach widzimy, że odchylenie standardowe odległości między punktami jest rzędu tych odległości, co swiadczy o dużym chaosie rozrzutu. W większych wymiarach odchylenie standardowe maleje w porównaniu do odległości, co oznacza że odległości między punktami są już (relatywnie) bliskie pewnej wartości dla danego wymiaru.

Mowa tu oczywiście o bliskosci relatywnej, gdyż warto zauwazyć, że spadająca wartość stosunku tych dwóch wielkości bierze się z tego, że (jak widać na wykresie wyżej), średnai odległość między punktami rośnie wraz z wymiarem znacznie szybciej, niż średnie odchylenie standardowe - ale obie te wielkości faktycznie się zwiększają.

## Kąty między wektorami w hiperprzestrzeni
**Ponownie mamy losowo zapełniony punktami hipersześcian o krawędziach długości 1. Z tych punktowych losujemy (bez zwracania) dwie pary. Punkty z pary wyznaczają pewien wektor (są jego początkiem i końcem). Jaki jest kąt między dwoma wylosowanymi punktami? Losowanie powtórz wielokrotnie. Jak wygląda rozkład otrzymanych kątów?**

In [ ]:
Vector = Tuple[np.ndarray, np.ndarray]

def length(v: Vector) -> float:
    v_abs = v[1] - v[0]
    return np.sqrt((v_abs ** 2).sum())

def angle(
    v1: Vector,
    v2: Vector,
) -> float:
    v1_abs = v1[1] - v1[0]
    v2_abs = v2[1] - v2[0]
    cos = (v1_abs * v2_abs).sum() / (length(v1) * length(v2))
    return np.arccos(cos)

In [ ]:
def hyper_angles(degree: int, X: float=1.0, n_samples: int = 5000) -> np.ndarray:
    n_points = 4 * n_samples
    p = np.random.rand(n_points, degree)
    return np.array([
        angle(
            (p[4 * i], p[4 * i + 1]),
            (p[4 * i + 2], p[4 * i + 3])
        )
        for i in range(n_samples)
    ])

In [ ]:
for i in range(10):
    degree = 2 ** i
    plt.hist(hyper_angles(degree), bins=100, stacked=True, density=True, range=(0, np.pi))
    print(f"degree: {degree}")
    plt.show()

#### Komentarz

Wraz ze wzrostem wymiaru danych, rozkład kątów, jaki tworzą dwa losowo wybrane wektory zawęża się i zaczyna koncentrować na $\dfrac{\pi}{2}$. Oznacza to niezależność takich wektorów, a więc i próżne próby doszukiwania się między nimi podobieństwa w sensie kątowym.